# Практическое задание.

## Постановка задач

Вам дан абстрактный базовый класс, формализующий интерфейс универсального регрессора - `RegressorABC`. RegressorABC имеет два абстрактных метода: fit и predict. Метод fit используется для извлечения информации из экспериментальных данных, эта информация используется для построения регрессионных зависимостей. Метод predict используется для получения предсказанного алгоритмом значения в заданной точке или заданном наборе точек. Используя абстрактный базовый класс, вам необходимо реализовать два конкретных регрессионных алгоритма, наследуя интерфейс регрессора.

**Задачи**:
- Реализуйте алгоритм непараметрической регресси, используя интерфейс, описанный в классе RegressorABC - решаем вместе;  
- Реализуйте Метод наименьших квадратов, используя интерфейс, описанный в классе RegressorABC - самостоятельно;   

## Теоретическая справка

### Общая постановка задачи регрессии

Заданы некоторые множества: $X$ - множество объектов, и $Y$ - множество ответов;  
$y: X \rightarrow Y$ - неизвестная зависимость;  
$X^l = \{x_1, x_2, ..., x_l\} \subset X$ - обучающая выборка;  
$y_i = y(x_i), i = 1, l$ - известные ответы для обучающей выборки;  

В рамках данного задания $X = \mathbb{R}$, $Y = \mathbb{R}$;  

**Задача** состоит в том, чтобы найти алгоритм $a: X \rightarrow Y$, приближающий $y$ на всём множестве $X$, используя обучающую выборку $(X^l, y^l)$;   
**Замечание**: $y$ - истинная зависимость, $a$ - аппроксимация.


### Непараметрическая Регрессия

В рамках непараметрической регрессии мы пытаемся приблизить значении $y(x)$ некоторой константой $\alpha$ в некоторой окрестности точки $x$. Задача сводится к нахождению данной константы для каждого $x \in X$.

Для того, чтобы отыскать оптимальное значения данной константы, решим следующую оптимизационную задачу:

$Q(\alpha, X^l) = \sum_{i = 1}^{l}{w_i(x)(\alpha - y_i)^2} \rightarrow min_{\alpha \in \mathbb{R}},$ где $w_i(x) = K(\frac{\rho(x, x_i)}{h})$ - вес i-ого квадратичного отклонения, $K(\frac{\rho(x, x_i)}{h})$ - ядро, невозрастающая, ограниченная, гладкая функция, $h$ - ширина окна сглаживания. Фактически мы минимизируем взвешанную сумму квадратов отклонений известных ответов от подобранной константы по $\alpha$.

$$
\frac{d}{d\alpha}Q(\alpha, X^l) = \sum_{i = 1}^{l}{w_i(x)(2\alpha - 2y_i)} = 0;  
\\ \sum_{i = 1}^{l}{\alpha w_i(x)} = \sum_{i = 1}^{l}{y_i w_i(x)} \rightarrow \alpha = \frac{\sum_{i = 1}^{l}{y_i w_i(x)}}{\sum_{i = 1}^{l}{w_i(x)}};
\\ a(x, X^l) = \frac{\sum_{i = 1}^{l}{y_i w_i(x)}}{\sum_{i = 1}^{l}{w_i(x)}} = \frac{\sum_{i = 1}^{l}{y_i K(\frac{\rho(x, x_i)}{h})}}{\sum_{i = 1}^{l}{K(\frac{\rho(x, x_i)}{h})}};
$$  

Таким образом мы получили аппроксимацию $a(x, X^l) = \frac{\sum_{i = 1}^{l}{y_i K(\frac{\rho(x, x_i)}{h})}}{\sum_{i = 1}^{l}{K(\frac{\rho(x, x_i)}{h})}}$, которая позволяет вычислять значение функции в данной точке по имеющимся данным.

В качестве ядра будем использовать так называемое [ядро Епанечникова](https://ru.wikipedia.org/wiki/%D0%AF%D0%B4%D1%80%D0%BE_(%D1%81%D1%82%D0%B0%D1%82%D0%B8%D1%81%D1%82%D0%B8%D0%BA%D0%B0)):
$$
\begin{equation*}
K(x) = 
 \begin{cases}
   \frac{3}{4}(1 - x^2), |x| \le 1\\
   0, |x| > 1
 \end{cases}
\end{equation*}
$$

Ещё одним тонким моментом, связанным с ядром, является ширина окна $h$.Ширина окна - некоторое число, которое можно подобрать руками, или реализовать адаптивный расчет. Т.к. адаптивный расчет является более надёжным методом, не требующим долгой настройки, мы пойдём именно по этому пути. 

Для адаптивного расчета ширины окна необходимо вычислить расстояния от элемента $x$, для которого мы хотим осуществить предсказание, до всех элементов $x_i \in X^l$ обучающей выборки. После чего необходимо упорядочить их по возрастанию расстояния до $x$. Из полученного вариационного ряда выберем k-ый элемент $x_k$. Расстояние $\rho(x, x_k)$ и будет нашей шириной окна, адаптивно вычисляемой для каждого элемента $x \in X$. Может показаться, что мы несильно упростили задачу, ведь теперь вместо выбора $h$ мы будем выбирать $k$. Но, благодаря подобным действиям мы получим лучшие результаты, ведь при данном подходе ширина окна будет своя для кажого $x$. 

___

## Дополнительные материалы.

1. [Машинное обучение. Метрические методы. К.В. Воронцов, Школа анализа данных, Яндекс.](https://www.youtube.com/watch?v=mCJcRDIY4TI&list=PLJOzdkh8T5krxc4HsHbB8g8f0hu7973fK&index=3&t=2079s)